# Teil 4 Demo 1: Abfragen mit SQL

## Bibliotheken & Konfiguration

In [ ]:
packages <- c("readr", "DBI", "RSQLite")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2021-Block-7/master/Rohdaten/mimic-iii-demo/"

## Ausgewählte MIMIC III-Rohdaten laden

In [ ]:
mimic.patients.raw      <- read_csv(paste(base_url, "PATIENTS.csv", sep=""),
                                    col_types = cols(row_id = col_double(), subject_id = col_double(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()))
mimic.admissions.raw    <- read_csv(paste(base_url, "ADMISSIONS.csv", sep=""),
                                    col_types = cols(  row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()))
mimic.prescriptions.raw <- read_csv(paste(base_url, "PRESCRIPTIONS.csv", sep=""),
                                    col_types = cols(row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), icustay_id = col_double(), startdate = col_datetime(format = ""), enddate = col_datetime(format = ""), drug_type = col_character(), drug = col_character(), drug_name_poe = col_character(), drug_name_generic = col_character(), formulary_drug_cd = col_character(), gsn = col_character(), ndc = col_character(), prod_strength = col_character(), dose_val_rx = col_character(), dose_unit_rx = col_character(), form_val_disp = col_character(), form_unit_disp = col_character(), route = col_character()))
head(mimic.patients.raw)
head(mimic.admissions.raw)
head(mimic.prescriptions.raw)

## Rohdaten in SQLite-Datenbank (im Speicher) laden

In [ ]:
con <- dbConnect(RSQLite::SQLite(), ":memory:")
dbWriteTable(con, "patients",      mimic.patients.raw)
dbWriteTable(con, "admissions",    mimic.admissions.raw)
dbWriteTable(con, "prescriptions", mimic.prescriptions.raw)
dbListTables(con)

## Alle Spalten einer Tabelle abfragen

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM patients
"), 6)

## Ausgewählte Spalten einer Tabelle abfragen

In [ ]:
head(dbGetQuery(con, "
SELECT subject_id,
       gender
  FROM patients
"))

## Ausgewählte Zeilen einer Tabelle abfragen

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM patients
 WHERE gender = 'M'
"))

## Ergebnis auf eindeutige Datensätze reduzieren (DISTINCT)

In [ ]:
head(dbGetQuery(con, "
SELECT DISTINCT
       gender
  FROM patients
"))

## Einfache Aggregation: Gesamtzahl der Datensätze einer Tabelle abfragen

In [ ]:
head(dbGetQuery(con, "
SELECT COUNT(*) AS n
  FROM patients
"))

## Aggregation & Gruppierung: Anzahl nach Geschlecht ermitteln

In [ ]:
head(dbGetQuery(con, "
SELECT gender,
       COUNT(*) AS n
  FROM patients
 GROUP BY gender
"))

## Zeilen nach der Aggregation filtern

In [ ]:
head(dbGetQuery(con, "
SELECT gender,
       COUNT(*) AS n
  FROM patients
 GROUP BY gender
HAVING COUNT(*) > 50
"))

## Tabellen per JOIN miteinander verknüpfen

In [ ]:
head(dbGetQuery(con, "
SELECT pat.subject_id,
       pat.gender,
       adm.diagnosis
  FROM patients   pat
  JOIN admissions adm ON pat.subject_id = adm.subject_id
"))

## Patient:innen mit mehr als einem Intensiv-Aufenthalt abfragen

In [ ]:
head(dbGetQuery(con, "
SELECT pat.subject_id,
       COUNT(*) AS n_admissions
  FROM patients   pat
  JOIN admissions adm ON pat.subject_id = adm.subject_id
 GROUP BY pat.subject_id
HAVING COUNT(*) > 1
 ORDER BY COUNT(*) DESC
"))

## 2 Subsets von Verschreibungen für die Demo verschiedener Joins erzeugen

* Patient:innenen mit Hauptdiagnose Sepsis
* Patient:innen mit Gabe von Vancomycin (Reserve-Antibiotikum z.B. bei MRSA)

In [ ]:
dbExecute(con, "CREATE TABLE demo_sepsis     AS SELECT DISTINCT subject_id FROM admissions    WHERE LOWER(diagnosis) LIKE '%sepsis%'")
dbExecute(con, "CREATE TABLE demo_vancomycin AS SELECT DISTINCT subject_id FROM prescriptions WHERE LOWER(drug)      LIKE '%vancomycin%'")

## Patient:innen abfragen, die sowohl eine Sepsis als Hauptdiagnose hatten als auch Vancomycin erhalten haben (INNER JOIN)

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM demo_sepsis     sep
  JOIN demo_vancomycin van on sep.subject_id = van.subject_id
"), 100)

## Alle Patient:innen abfragen, die eine Sepsis als Hauptdiagnose hatten sowie die mit Vancomycingabe ergänzen (LEFT OUTER JOIN)

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM      demo_sepsis     sep
  LEFT JOIN demo_vancomycin van on sep.subject_id = van.subject_id
"), 100)

## Alle Patient:innen abfragen, die eine Vancomycingabe hatten und die keine Sepsisdiagnose hatten (RIGHT OUTER JOIN)

### Hinweis: SQLite unterstützt keine RIGHT JOINS, deshalb ist das hier als LEFT JOIN mit vertauschter Reihenfolge dargestellt

In [ ]:
head(dbGetQuery(con, "
SELECT *
  FROM      demo_vancomycin van
  LEFT JOIN demo_sepsis     sep on van.subject_id = sep.subject_id
"), 100)

## Tatsächliche Hauptdiagnosen für die Patient:innen mit Vancomycingabe ohne Sepsis ermitteln

In [ ]:
head(dbGetQuery(con, "
SELECT adm.diagnosis,
       COUNT(*)
  FROM      demo_vancomycin van
  JOIN      admissions      adm ON van.subject_id = adm.subject_id
  LEFT JOIN demo_sepsis     sep on van.subject_id = sep.subject_id
 WHERE sep.subject_id IS NULL
 GROUP BY adm.diagnosis
 ORDER BY COUNT(*) DESC
"), 20)

## Datenbankverbindung schließen

In [ ]:
dbDisconnect(con)